In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')

from datetime import timedelta

In [2]:
from config_paper import paper_key_id, paper_secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(paper_key_id, paper_secret_key,base_url)

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-05-14'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        
        dataframe['emaclose'] = tab.EMA(dataframe, timeperiod=4, price='close')
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['emapulse'] = ta.RSI(dataframe['emaclose'], timeperiod=3)
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14, price='close')

        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.cash

ACTIVE


'100000'

In [7]:
symbollist=pd.read_csv('nasdaq-100-index-05-06-2020.csv')['Symbol']

In [8]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [9]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
#assetsToTrade = symbollist
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.03

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

In [10]:
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
            
    #asset Manager
    assetlist=api.list_positions()
    
    for asset in assetlist:
            #if asset is in list of positions AND is more than 2.4% loss
            # sell and turn that symbol to null
        if float(asset.unrealized_intraday_plpc)< -.024 :
            openPosition = api.get_position(asset.symbol)
            orderlist=api.list_orders()
            
            
            for order in orderlist:
                if order.symbol==asset.symbol:
                    if order.status=='new':
                        if order.side=='sell':
                            ridorder=api.cancel_order(order.id)
                        
            returned = api.submit_order(asset.symbol,int(openPosition.qty),"sell","market","day")
            j=assetsToTrade.index(asset.symbol)
            assetsToTrade[j]= asset.symbol+' nulled for below -2.4% pnl ' +str(currenttime)
            print('Sell', asset.symbol)
            
            
    #buy and sell        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                #api.get_barset(symbol,barTimeframe,limit=150).df
            data=populateindicators(returned_data)
        except:
            print(symbol)
        
        
        
#buy conditions -------
        try:
        #set last time to buy stocks and when to start buying
            if currenttime <=1230 and currenttime >=639:    


            # Calculated trading indicators
            #populate exponential moving averages




             # Create the buy signal here
                if data['emapulse'][-2] > data['emapulse'][-3]+35 and \
                data['adx'][-3] < 20 and \
                data['high'][-1]-data['close'][-1]+.02 < data['close'][-1]-data['low'][-1] and \
                data['close'][-1] > data['open'][-1]+.02:
                    buyord=True
                    assetlist=api.list_positions()
                    for asset in assetlist:
                        # if asset is in the list of positions we have already bought, then do not buy again
                        if asset.symbol == symbol:
                            buyord=False
                            print(asset.symbol,asset.qty)







                    #if asset is not already bought, then buy             
                    if buyord==True:
                        #cashBalance = account.daytrading_buying_power
                        #equity
                        cashBalance = float(account.buying_power)

                       

                        targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))
                        buyreturned = api.submit_order(symbol,targetPositionSize,"buy","limit","day",limit_price=data['close'][-1]+.05)
                        print('Buy ',buyreturned)
        except:
            print('error with ',symbol)

    
    #get current time and shut off trading after 12
     
    if currenthour >= 13: 
        trade=False
        print('Good work, Turning off')
    

    
    

AGN
BRKB
BFB
MKTK
RTN
UTX

AGN
BRKB
BFB
MKTK
RTN
UTX

AGN
BRKB
BFB
MKTK
RTN
UTX

AGN
BRKB
BFB
MKTK
RTN
UTX

AGN
BRKB
BFB
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'e546b55a-77d5-4e6c-b588-0aeaa20bd2b3',
    'canceled_at': None,
    'client_order_id': 'd80ef51f-2ea1-41e4-98e4-b7fbc42260e8',
    'created_at': '2020-05-14T18:41:59.832384Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'eec6a207-5973-479b-a2ec-66bb72befea0',
    'legs': None,
    'limit_price': '206.64',
    'order_class': '',
    'order_type': 'limit',
    'qty': '19',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-14T18:41:59.820696Z',
    'symbol': 'CLX',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-14T18:41:59.832384Z'})
MKTK
RTN
UTX

A

RTN
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'b2b91a9b-3ecd-45ed-bf53-1f7d0613e1e8',
    'canceled_at': None,
    'client_order_id': 'a0d134a4-d528-4eb5-8ca2-9346aea6788f',
    'created_at': '2020-05-14T18:46:55.29314Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '66b728a1-dc8a-4fd3-a907-a0885d6ce7f6',
    'legs': None,
    'limit_price': '106.14',
    'order_class': '',
    'order_type': 'limit',
    'qty': '37',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-14T18:46:55.269378Z',
    'symbol': 'SWKS',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-14T18:46:55.29314Z'})
UTX

AGN
BRKB
BFB
MKTK
RTN
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '2e65dd16-9fd7-42c7-aa0f-bd7950f028be',
    'ca

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '713c63c2-aa51-449b-bc1d-cb5ef9a7d4ff',
    'canceled_at': None,
    'client_order_id': '080902dd-6836-4f08-b04d-78f999f580ef',
    'created_at': '2020-05-14T18:53:26.749647Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '865aec17-8b14-4ea0-8ab9-351ff1685d9c',
    'legs': None,
    'limit_price': '61.24',
    'order_class': '',
    'order_type': 'limit',
    'qty': '65',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-14T18:53:26.696273Z',
    'symbol': 'RJF',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-14T18:53:26.749647Z'})
RTN
error with  RTN
UTX

AGN
BRKB
BFB
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '19020a7c-7741-401b-8422-37ad25d044b7',


RTN
UTX

AGN
BRKB
BFB
MKTK
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'ac20a75d-5bb4-4d10-96e8-f5e58e730139',
    'canceled_at': None,
    'client_order_id': '6e4fab48-3c0e-4a6c-bc28-b5cb2e1d5a7d',
    'created_at': '2020-05-14T19:10:58.726361Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '5cfb0290-75b2-4852-99ce-40cc061cc96d',
    'legs': None,
    'limit_price': '231.23',
    'order_class': '',
    'order_type': 'limit',
    'qty': '17',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-14T19:10:58.679771Z',
    'symbol': 'NEE',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-14T19:10:58.726361Z'})
RTN
UTX

AGN
BRKB
BFB
MKTK
RTN
UTX

AGN
BRKB
BFB
MKTK
RTN
UTX

AGN
BRKB
BFB
MKTK
RTN
UTX

AGN
BRKB
BFB
MKTK
R

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [ ]:
data['emapulse'][-3]